<a href="https://colab.research.google.com/github/daisuke08253649/DeepLearning/blob/main/Reduction_of_default_risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install japanize_matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

train_df = pd.read_csv('./drive/MyDrive/DeepLearning/SIGNATE/Reduction_of_default_risk/train3.csv')
train_df = train_df.drop(columns=['id'])
train_df = train_df.dropna()
print(train_df.head())
print(train_df.isnull().sum())
print(train_df.info())

In [ ]:
loan_amnt = train_df['loan_amnt']
loan_amnt.plot.hist(title='loan_amnt')
plt.show()

In [ ]:
interest_rate = train_df['interest_rate']
interest_rate.plot.hist(title='interest_rate')
plt.show()

In [ ]:
employment_length_counts = train_df['employment_length'].value_counts()
print(employment_length_counts)

In [ ]:
employment_length_counts.plot.bar(title='employment_length_counts')
plt.xlabel('employment_length')
plt.ylabel('count')
plt.show()

In [ ]:
train_df['loan_status'].value_counts().plot.bar()
plt.show()

In [ ]:
sns.boxplot(x='loan_status', y='loan_amnt', data=train_df)
plt.title('loan_status vs loan_amnt')
plt.xlabel('loan_status')
plt.ylabel('loan_amnt')
plt.show()

In [ ]:
sns.boxplot(x='loan_status', y='interest_rate', data=train_df)
plt.title('loan_status vs interest_rate')
plt.xlabel('loan_status')
plt.ylabel('interest_rate')
plt.show()

In [ ]:
sns.boxplot(x='loan_status', y='credit_score', data=train_df)
plt.title('loan_status vs credit_score')
plt.xlabel('loan_status')
plt.ylabel('credit_score')
plt.show()

In [ ]:
cross_term = pd.crosstab(train_df['term'], train_df['loan_status'], margins=True)
c_rate = cross_term['ChargedOff'] / cross_term['All']
f_rate = cross_term['FullyPaid'] / cross_term['All']
cross_term['c_rate'] = c_rate
cross_term['f_rate'] = f_rate
cross_term = cross_term.drop(index='All')
print(cross_term)

df_bar = cross_term[['c_rate', 'f_rate']]
df_bar.plot.bar(stacked=True)
plt.title('返済期間ごとの貸し倒れ率と完済率')
plt.xlabel('期間')
plt.ylabel('割合')
plt.show()

In [ ]:
cross_el = pd.crosstab(train_df['employment_length'], train_df['loan_status'], margins=True)
c_rate = cross_el['ChargedOff'] / cross_el['All']
f_rate = cross_el['FullyPaid'] / cross_el['All']
cross_el['c_rate'] = c_rate
cross_el['f_rate'] = f_rate
cross_el = cross_el.drop(index='All')
print(cross_el)

df_bar = cross_el[['c_rate', 'f_rate']]
df_bar.plot.bar(stacked=True)
plt.title('勤続年数ごとの貸し倒れ率と完済率')
plt.xlabel('勤続年数')
plt.ylabel('割合')
plt.show()

In [ ]:
cross_grade = pd.crosstab(train_df['grade'], train_df['loan_status'], margins=True)
c_rate = cross_grade['ChargedOff'] / cross_grade['All']
f_rate = cross_grade['FullyPaid'] / cross_grade['All']
cross_grade['c_rate'] = c_rate
cross_grade['f_rate'] = f_rate
cross_grade = cross_grade.drop(index='All')
print(cross_grade)

df_bar = cross_grade[['c_rate', 'f_rate']]
df_bar.plot.bar(stacked=True)
plt.title('グレードごとの貸し倒れ率と完済率')
plt.xlabel('グレード')
plt.ylabel('割合')
plt.show()

In [ ]:
cross_purpose = pd.crosstab(train_df['purpose'], train_df['loan_status'], margins=True)
c_rate = cross_purpose['ChargedOff'] / cross_purpose['All']
f_rate = cross_purpose['FullyPaid'] / cross_purpose['All']
cross_purpose['c_rate'] = c_rate
cross_purpose['f_rate'] = f_rate
cross_purpose = cross_purpose.drop(index='All')
print(cross_purpose)

df_bar = cross_purpose[['c_rate', 'f_rate']]
df_bar.plot.bar(stacked=True)
plt.title('目的ごとの貸し倒れ率と完済率')
plt.xlabel('目的')
plt.ylabel('割合')
plt.show()

In [ ]:
cross_at = pd.crosstab(train_df['application_type'], train_df['loan_status'], margins=True)
c_rate = cross_at['ChargedOff'] / cross_at['All']
f_rate = cross_at['FullyPaid'] / cross_at['All']
cross_at['c_rate'] = c_rate
cross_at['f_rate'] = f_rate
cross_at = cross_at.drop(index='All')
print(cross_at)

df_bar = cross_at[['c_rate', 'f_rate']]
df_bar.plot.bar(stacked=True)
plt.title('目的ごとの貸し倒れ率と完済率')
plt.xlabel('目的')
plt.ylabel('割合')
plt.show()

In [ ]:
x = train_df[['term', 'interest_rate', 'grade', 'purpose', 'credit_score', 'application_type']]
label = train_df['loan_status']

x_dum = pd.get_dummies(x)
label_dum = pd.get_dummies(label)
label_dum = label_dum.drop(columns='FullyPaid')
print(x_dum.head())
print(label_dum.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix

x_train, x_test, y_train, y_test = train_test_split(x_dum, label_dum, random_state=0, stratify=label_dum['ChargedOff'])

model = LogisticRegression()
model.fit(x_train, y_train)

pred = model.predict(x_test)
f_one = f1_score(y_test, pred)
print(f_one)
cm = confusion_matrix(y_test, pred)
print(cm)

In [ ]:
model2 = LogisticRegression()
model2.fit(x_train, y_train)
pred_proba = model2.predict_proba(x_test)
proba_df = pd.DataFrame(pred_proba)
#閾値の変更

#「貸し倒れ」の数の合計を計算
chargedoff_val = y_test['ChargedOff'].sum()
#print(chargedoff_val)

#proba_dfデータフレームの2列目の数値を降順（数値が大きい順）に並べ替える。
proba_desc = proba_df[1].sort_values(ascending=False)
#print(proba_desc)

#proba_descからchargedoff_valで取得した数値と一致する行数を取得する。（proba_descのchargedoff_val番目の数値を取得）
threshold = proba_desc.iloc[chargedoff_val]
#print(threshold)

def classification(x):
    if x >= threshold:
        return 1
    else:
        return 0

proba_df['class_new'] = proba_df[1].apply(classification)
#print(proba_df.head())

f_one_2 = f1_score(y_test, proba_df['class_new'])
print(f_one_2)
cm2 = confusion_matrix(y_test, proba_df['class_new'])
print(cm2)

In [ ]:
#ダウンサンプリング


#print(x_train.shape)
train = pd.concat([x_train, y_train], axis=1)
#print(train.shape)

fp = train[train['ChargedOff']==0]
co = train[train['ChargedOff']==1]
#print(co.shape[0])
fp = fp.sample(n=co.shape[0], random_state=0)
#print(fp.shape[0])

train = pd.concat([fp, co])
x_train = train.drop(columns='ChargedOff')
#print(x_train.head())
y_train = train['ChargedOff']
#print(y_train.head())


model3 = LogisticRegression()
model3.fit(x_train, y_train)
pred3 = model3.predict(x_test)
print(pred3)
f_one_3 = f1_score(y_test, pred3)
print(f_one_3)
cm3 = confusion_matrix(y_test, pred3)
print(cm3)

In [ ]:
test_df = pd.read_csv('./drive/MyDrive/DeepLearning/SIGNATE/Reduction_of_default_risk/test2.csv')
test_df = test_df.drop(columns=['id'])
#test_df = test_df.dropna()
print(test_df.head())
print(test_df.isnull().sum())
print(test_df.info())
print(len(test_df))

In [ ]:
submit_df = pd.read_csv('./drive/MyDrive/DeepLearning/SIGNATE/Reduction_of_default_risk/submit2.csv')
print(len(submit_df))

In [ ]:
if len(test_df) > len(submit_df):
    test_df.drop(test_df.index[-1], inplace=True)

print(len(test_df))

In [ ]:
test = test_df[['term', 'interest_rate', 'grade', 'purpose', 'credit_score', 'application_type']]
test_dum = pd.get_dummies(test)
print(test_dum.head())

In [ ]:
test_pred = model3.predict(test_dum)
submit_df.iloc[:, 1] = test_pred
print(test_pred)
print(submit_df.head())

In [ ]:
submit_df.to_csv('./drive/MyDrive/DeepLearning/SIGNATE/Reduction_of_default_risk/submit2.csv', index=False)